In [1]:
import os

os.environ['PYTHONHASHSEED']=str(2)

import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import initializers
from keras import backend
from sklearn.preprocessing import MinMaxScaler
import joblib

#PARAMETERS
#samples = 200000 #number of PDF samples
samples = 150000
seq_length = 25 #sequence window to be used
batch_size = 128
molecules = 2
species = 'hmf'
downsample_factor = 10

#https://www.pythonfixing.com/2022/01/fixed-tesnorflow-20-tfrandomsetseed-not.html
def reset_random_seeds():
    os.environ['PYTHONHASHSEED']=str(2)
    tf.random.set_seed(28)
    np.random.seed(28)
    random.seed(28)

#reset_random_seeds()

data_1 = np.load('C:/Users/mali19/Downloads/KevinCode/inputs/input_latest/%s_data.npy' %species)
data = np.add(data_1, 3.549)
data = data[:samples:downsample_factor, :]
print(np.shape(data))
#scaler = MinMaxScaler()
#data = scaler.fit_transform(data)
#joblib.dump(scaler, '%s_scaler.gz' %species)




(15000, 6)


In [2]:
data_1[180000,:]

array([1.146, 1.063, 0.697, 1.327, 1.289, 0.783])

In [3]:
def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length):
        x = data[i:(i+seq_length), :]
        y = data[i+seq_length, :]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

input_, output = create_sequences(data, seq_length)

input_ = np.array(input_)
output = np.array(output)

indices = np.arange(len(input_))
np.random.shuffle(indices)

shuffled_data = input_[indices]
shuffled_targets = output[indices]

split_ratio = 0.80
split_index = int(len(input_)*split_ratio)

x1_train = shuffled_data[:split_index]
y1_train = shuffled_targets[:split_index]
x1_val = shuffled_data[split_index:]
y1_val = shuffled_targets[split_index:]



In [4]:
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)
    # Apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    # Apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

    # Feed Forward Part
    ffn = tf.keras.Sequential([
        layers.Dense(ff_dim, activation="relu"),
        layers.Dropout(dropout),
        layers.Dense(inputs.shape[-1]),
    ])
    x = ffn(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)
    return x

def build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_layers, dropout=0):
    inputs = layers.Input(shape=input_shape)
    x = inputs

    # Add positional encoding
    position_encoding = positional_encoding(input_shape[0], input_shape[1])
    x = x + position_encoding

    for _ in range(num_layers):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    # Selecting the last timestep output
    x = x[:, -1, :]  # This slices out the last timestep

    outputs = layers.Dense(molecules*3, activation='linear')(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model


# Model parameters
head_size = 256
num_heads = 4
ff_dim = 256
num_layers = 4
dropout_rate = 0.1

# Build the model
model = build_transformer_model(input_shape=(seq_length, molecules*3), head_size=head_size, num_heads=num_heads, ff_dim=ff_dim, num_layers=num_layers, dropout=dropout_rate)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mape')
model.summary()


callback = EarlyStopping(monitor="val_loss", min_delta=1e-5, patience=50, verbose=1, mode="min")
checkpoint = ModelCheckpoint(filepath='transformer_model_%s.h5' % species, save_best_only=True, monitor='val_loss', mode='min', verbose=1)

history = model.fit(x1_train, y1_train, callbacks=[callback, checkpoint], batch_size=batch_size, epochs=300, verbose=1, validation_data=(x1_val, y1_val), shuffle=False)
hist_df = pd.DataFrame(history.history)
hist_df.to_csv('11history_%s_4.csv' %species)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 25, 6)]              0         []                            
                                                                                                  
 tf.__operators__.add (TFOp  (None, 25, 6)                0         ['input_1[0][0]']             
 Lambda)                                                                                          
                                                                                                  
 multi_head_attention (Mult  (None, 25, 6)                27654     ['tf.__operators__.add[0][0]',
 iHeadAttention)                                                     'tf.__operators__.add[0][0]']
                                                                                             

                                                                                                  
 tf.__operators__.add_7 (TF  (None, 25, 6)                0         ['dropout_6[0][0]',           
 OpLambda)                                                           'layer_normalization_5[0][0]'
                                                                    ]                             
                                                                                                  
 layer_normalization_6 (Lay  (None, 25, 6)                12        ['tf.__operators__.add_7[0][0]
 erNormalization)                                                   ']                            
                                                                                                  
 sequential_3 (Sequential)   (None, 25, 6)                3334      ['layer_normalization_6[0][0]'
                                                                    ]                             
          

C:\Users\mali19\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


94/94 [==============================] - ETA: 0s - loss: 55.0469
Epoch 2: val_loss improved from 68.32247 to 39.85721, saving model to transformer_model_hmf.h5
94/94 [==============================] - 14s 147ms/step - loss: 55.0469 - val_loss: 39.8572
Epoch 3/300
94/94 [==============================] - ETA: 0s - loss: 26.3117
Epoch 3: val_loss improved from 39.85721 to 14.86799, saving model to transformer_model_hmf.h5
94/94 [==============================] - 14s 152ms/step - loss: 26.3117 - val_loss: 14.8680
Epoch 4/300
94/94 [==============================] - ETA: 0s - loss: 7.7839
Epoch 4: val_loss improved from 14.86799 to 2.32227, saving model to transformer_model_hmf.h5
94/94 [==============================] - 15s 155ms/step - loss: 7.7839 - val_loss: 2.3223
Epoch 5/300
94/94 [==============================] - ETA: 0s - loss: 0.9629
Epoch 5: val_loss improved from 2.32227 to 0.60028, saving model to transformer_model_hmf.h5
94/94 [==============================] - 14s 152ms/step

94/94 [==============================] - ETA: 0s - loss: 0.1997
Epoch 36: val_loss did not improve from 0.17248
94/94 [==============================] - 21s 219ms/step - loss: 0.1997 - val_loss: 0.1830
Epoch 37/300
94/94 [==============================] - ETA: 0s - loss: 0.1975
Epoch 37: val_loss improved from 0.17248 to 0.16666, saving model to transformer_model_hmf.h5
94/94 [==============================] - 20s 212ms/step - loss: 0.1975 - val_loss: 0.1667
Epoch 38/300
94/94 [==============================] - ETA: 0s - loss: 0.1915
Epoch 38: val_loss improved from 0.16666 to 0.15821, saving model to transformer_model_hmf.h5
94/94 [==============================] - 20s 208ms/step - loss: 0.1915 - val_loss: 0.1582
Epoch 39/300
94/94 [==============================] - ETA: 0s - loss: 0.1948
Epoch 39: val_loss did not improve from 0.15821
94/94 [==============================] - 19s 204ms/step - loss: 0.1948 - val_loss: 0.1820
Epoch 40/300
94/94 [==============================] - ETA: 0s

...

94/94 [==============================] - ETA: 0s - loss: 0.1024
Epoch 144: val_loss did not improve from 0.08371
94/94 [==============================] - 26s 274ms/step - loss: 0.1024 - val_loss: 0.0886
Epoch 145/300
94/94 [==============================] - ETA: 0s - loss: 0.1017
Epoch 145: val_loss did not improve from 0.08371
94/94 [==============================] - 30s 322ms/step - loss: 0.1017 - val_loss: 0.0916
Epoch 146/300
94/94 [==============================] - ETA: 0s - loss: 0.1025
Epoch 146: val_loss did not improve from 0.08371
94/94 [==============================] - 22s 227ms/step - loss: 0.1025 - val_loss: 0.0855
Epoch 147/300
94/94 [==============================] - ETA: 0s - loss: 0.1053
Epoch 147: val_loss did not improve from 0.08371
94/94 [==============================] - 16s 169ms/step - loss: 0.1053 - val_loss: 0.0872
Epoch 148/300
94/94 [==============================] - ETA: 0s - loss: 0.0999
Epoch 148: val_loss did not improve from 0.08371
94/94 [============

94/94 [==============================] - ETA: 0s - loss: 0.0952
Epoch 181: val_loss did not improve from 0.07759
94/94 [==============================] - 24s 252ms/step - loss: 0.0952 - val_loss: 0.0933
Epoch 182/300
94/94 [==============================] - ETA: 0s - loss: 0.0975
Epoch 182: val_loss did not improve from 0.07759
94/94 [==============================] - 21s 223ms/step - loss: 0.0975 - val_loss: 0.0840
Epoch 183/300
94/94 [==============================] - ETA: 0s - loss: 0.0955
Epoch 183: val_loss improved from 0.07759 to 0.07758, saving model to transformer_model_hmf.h5
94/94 [==============================] - 22s 235ms/step - loss: 0.0955 - val_loss: 0.0776
Epoch 184/300
94/94 [==============================] - ETA: 0s - loss: 0.0937
Epoch 184: val_loss did not improve from 0.07758
94/94 [==============================] - 20s 209ms/step - loss: 0.0937 - val_loss: 0.0806
Epoch 185/300
94/94 [==============================] - ETA: 0s - loss: 0.0936
Epoch 185: val_loss di

94/94 [==============================] - 19s 206ms/step - loss: 0.0917 - val_loss: 0.0820
Epoch 219/300
94/94 [==============================] - ETA: 0s - loss: 0.0898
Epoch 219: val_loss did not improve from 0.07737
94/94 [==============================] - 19s 204ms/step - loss: 0.0898 - val_loss: 0.0797
Epoch 220/300
94/94 [==============================] - ETA: 0s - loss: 0.0920
Epoch 220: val_loss did not improve from 0.07737
94/94 [==============================] - 19s 201ms/step - loss: 0.0920 - val_loss: 0.0836
Epoch 221/300
94/94 [==============================] - ETA: 0s - loss: 0.0936
Epoch 221: val_loss did not improve from 0.07737
94/94 [==============================] - 19s 198ms/step - loss: 0.0936 - val_loss: 0.0834
Epoch 222/300
94/94 [==============================] - ETA: 0s - loss: 0.0897
Epoch 222: val_loss improved from 0.07737 to 0.07434, saving model to transformer_model_hmf.h5
94/94 [==============================] - 19s 202ms/step - loss: 0.0897 - val_loss: 0.0

94/94 [==============================] - ETA: 0s - loss: 0.0897
Epoch 256: val_loss improved from 0.07434 to 0.07402, saving model to transformer_model_hmf.h5
94/94 [==============================] - 19s 201ms/step - loss: 0.0897 - val_loss: 0.0740
Epoch 257/300
94/94 [==============================] - ETA: 0s - loss: 0.0941
Epoch 257: val_loss did not improve from 0.07402
94/94 [==============================] - 19s 198ms/step - loss: 0.0941 - val_loss: 0.0759
Epoch 258/300
94/94 [==============================] - ETA: 0s - loss: 0.0906
Epoch 258: val_loss did not improve from 0.07402
94/94 [==============================] - 19s 205ms/step - loss: 0.0906 - val_loss: 0.0810
Epoch 259/300
94/94 [==============================] - ETA: 0s - loss: 0.0890
Epoch 259: val_loss did not improve from 0.07402
94/94 [==============================] - 19s 206ms/step - loss: 0.0890 - val_loss: 0.0784
Epoch 260/300
94/94 [==============================] - ETA: 0s - loss: 0.0973
Epoch 260: val_loss im

94/94 [==============================] - ETA: 0s - loss: 0.0884
Epoch 293: val_loss did not improve from 0.07207
94/94 [==============================] - 19s 199ms/step - loss: 0.0884 - val_loss: 0.0818
Epoch 294/300
94/94 [==============================] - ETA: 0s - loss: 0.0875
Epoch 294: val_loss did not improve from 0.07207
94/94 [==============================] - 19s 201ms/step - loss: 0.0875 - val_loss: 0.0808
Epoch 295/300
94/94 [==============================] - ETA: 0s - loss: 0.0882
Epoch 295: val_loss did not improve from 0.07207
94/94 [==============================] - 19s 199ms/step - loss: 0.0882 - val_loss: 0.0768
Epoch 296/300
94/94 [==============================] - ETA: 0s - loss: 0.0877
Epoch 296: val_loss did not improve from 0.07207
94/94 [==============================] - 19s 202ms/step - loss: 0.0877 - val_loss: 0.0752
Epoch 297/300
94/94 [==============================] - ETA: 0s - loss: 0.1006
Epoch 297: val_loss did not improve from 0.07207
94/94 [============

In [9]:
# def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
#     # Attention and Normalization
#     x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
#     x = layers.Dropout(dropout)(x)
#     x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)

#     # Feed Forward Part
#     ffn = tf.keras.Sequential([
#         layers.Dense(ff_dim, activation="relu"),
#         layers.Dropout(dropout),
#         layers.Dense(inputs.shape[-1]),
#     ])
#     x = ffn(x)
#     x = layers.LayerNormalization(epsilon=1e-6)(x + inputs)
#     return x

# def build_transformer_model(input_shape, head_size, num_heads, ff_dim, num_layers, dropout=0):
#     inputs = layers.Input(shape=input_shape)
#     x = inputs

#     for _ in range(num_layers):
#         x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

#     # Selecting the last timestep output
#     x = x[:, -1, :]  # This slices out the last timestep

#     outputs = layers.Dense(molecules*3, activation='linear')(x)
#     model = Model(inputs=inputs, outputs=outputs)
#     return model

# # Model parameters
# head_size = 256
# num_heads = 4
# ff_dim = 256
# num_layers = 4
# dropout_rate = 0.1

# # Build the model
# model = build_transformer_model(input_shape=(seq_length, molecules*3), head_size=head_size, num_heads=num_heads, ff_dim=ff_dim, num_layers=num_layers, dropout=dropout_rate)
# model.compile(optimizer=Adam(learning_rate=0.001), loss='mape')
# model.summary()


# callback = EarlyStopping(monitor="val_loss", min_delta=1e-5, patience=50, verbose=1, mode="min")
# checkpoint = ModelCheckpoint(filepath='transformer_model_%s.h5' % species, save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# history = model.fit(x1_train, y1_train, callbacks=[callback, checkpoint], batch_size=batch_size, epochs=300, verbose=1, validation_data=(x1_val, y1_val), shuffle=False)
# hist_df = pd.DataFrame(history.history)
# hist_df.to_csv('11history_%s_4.csv' %species)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 25, 6)]              0         []                            
                                                                                                  
 multi_head_attention (Mult  (None, 25, 6)                27654     ['input_1[0][0]',             
 iHeadAttention)                                                     'input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 25, 6)                0         ['multi_head_attention[0][0]']
                                                                                                  
 tf.__operators__.add (TFOp  (None, 25, 6)                0         ['dropout[0][0]',        

                                                                    ]                             
                                                                                                  
 layer_normalization_6 (Lay  (None, 25, 6)                12        ['tf.__operators__.add_6[0][0]
 erNormalization)                                                   ']                            
                                                                                                  
 sequential_3 (Sequential)   (None, 25, 6)                3334      ['layer_normalization_6[0][0]'
                                                                    ]                             
                                                                                                  
 tf.__operators__.add_7 (TF  (None, 25, 6)                0         ['sequential_3[0][0]',        
 OpLambda)                                                           'layer_normalization_5[0][0]'
          

C:\Users\mali19\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


94/94 [==============================] - ETA: 0s - loss: 50.4596
Epoch 2: val_loss improved from 63.15056 to 37.66280, saving model to transformer_model_hmf.h5
94/94 [==============================] - 14s 151ms/step - loss: 50.4596 - val_loss: 37.6628
Epoch 3/300
94/94 [==============================] - ETA: 0s - loss: 25.4364
Epoch 3: val_loss improved from 37.66280 to 13.37641, saving model to transformer_model_hmf.h5
94/94 [==============================] - 14s 149ms/step - loss: 25.4364 - val_loss: 13.3764
Epoch 4/300
94/94 [==============================] - ETA: 0s - loss: 7.0835
Epoch 4: val_loss improved from 13.37641 to 1.71371, saving model to transformer_model_hmf.h5
94/94 [==============================] - 14s 150ms/step - loss: 7.0835 - val_loss: 1.7137
Epoch 5/300
94/94 [==============================] - ETA: 0s - loss: 0.7868
Epoch 5: val_loss improved from 1.71371 to 0.47778, saving model to transformer_model_hmf.h5
94/94 [==============================] - 14s 151ms/step

94/94 [==============================] - 20s 208ms/step - loss: 0.2416 - val_loss: 0.2312
Epoch 71/300
94/94 [==============================] - ETA: 0s - loss: 0.2436
Epoch 71: val_loss improved from 0.22846 to 0.22771, saving model to transformer_model_hmf.h5
94/94 [==============================] - 19s 202ms/step - loss: 0.2436 - val_loss: 0.2277
Epoch 72/300
94/94 [==============================] - ETA: 0s - loss: 0.2397
Epoch 72: val_loss improved from 0.22771 to 0.22662, saving model to transformer_model_hmf.h5
94/94 [==============================] - 19s 202ms/step - loss: 0.2397 - val_loss: 0.2266
Epoch 73/300
94/94 [==============================] - ETA: 0s - loss: 0.2408
Epoch 73: val_loss improved from 0.22662 to 0.22597, saving model to transformer_model_hmf.h5
94/94 [==============================] - 19s 204ms/step - loss: 0.2408 - val_loss: 0.2260
Epoch 74/300
94/94 [==============================] - ETA: 0s - loss: 0.2391
Epoch 74: val_loss did not improve from 0.22597
94

94/94 [==============================] - 20s 215ms/step - loss: 0.2131 - val_loss: 0.1972
Epoch 106/300
94/94 [==============================] - ETA: 0s - loss: 0.2104
Epoch 106: val_loss did not improve from 0.19723
94/94 [==============================] - 19s 200ms/step - loss: 0.2104 - val_loss: 0.2110
Epoch 107/300
94/94 [==============================] - ETA: 0s - loss: 0.2122
Epoch 107: val_loss did not improve from 0.19723
94/94 [==============================] - 19s 205ms/step - loss: 0.2122 - val_loss: 0.2025
Epoch 108/300
94/94 [==============================] - ETA: 0s - loss: 0.2111
Epoch 108: val_loss did not improve from 0.19723
94/94 [==============================] - 19s 199ms/step - loss: 0.2111 - val_loss: 0.2001
Epoch 109/300
94/94 [==============================] - ETA: 0s - loss: 0.2092
Epoch 109: val_loss improved from 0.19723 to 0.19595, saving model to transformer_model_hmf.h5
94/94 [==============================] - 19s 201ms/step - loss: 0.2092 - val_loss: 0.1

94/94 [==============================] - ETA: 0s - loss: 0.1893
Epoch 141: val_loss did not improve from 0.17460
94/94 [==============================] - 19s 205ms/step - loss: 0.1893 - val_loss: 0.1766
Epoch 142/300
94/94 [==============================] - ETA: 0s - loss: 0.1874
Epoch 142: val_loss did not improve from 0.17460
94/94 [==============================] - 19s 205ms/step - loss: 0.1874 - val_loss: 0.1774
Epoch 143/300
94/94 [==============================] - ETA: 0s - loss: 0.1839
Epoch 143: val_loss improved from 0.17460 to 0.17457, saving model to transformer_model_hmf.h5
94/94 [==============================] - 19s 206ms/step - loss: 0.1839 - val_loss: 0.1746
Epoch 144/300
94/94 [==============================] - ETA: 0s - loss: 0.1845
Epoch 144: val_loss did not improve from 0.17457
94/94 [==============================] - 19s 203ms/step - loss: 0.1845 - val_loss: 0.1748
Epoch 145/300
94/94 [==============================] - ETA: 0s - loss: 0.1852
Epoch 145: val_loss im

Epoch 213/300
94/94 [==============================] - ETA: 0s - loss: 0.1635
Epoch 213: val_loss did not improve from 0.14896
94/94 [==============================] - 19s 205ms/step - loss: 0.1635 - val_loss: 0.1516
Epoch 214/300
94/94 [==============================] - ETA: 0s - loss: 0.1595
Epoch 214: val_loss did not improve from 0.14896
94/94 [==============================] - 19s 206ms/step - loss: 0.1595 - val_loss: 0.1534
Epoch 215/300
94/94 [==============================] - ETA: 0s - loss: 0.1599
Epoch 215: val_loss did not improve from 0.14896
94/94 [==============================] - 19s 203ms/step - loss: 0.1599 - val_loss: 0.1578
Epoch 216/300
94/94 [==============================] - ETA: 0s - loss: 0.1584
Epoch 216: val_loss improved from 0.14896 to 0.14881, saving model to transformer_model_hmf.h5
94/94 [==============================] - 20s 209ms/step - loss: 0.1584 - val_loss: 0.1488
Epoch 217/300
94/94 [==============================] - ETA: 0s - loss: 0.1587
Epoch 21

Epoch 250/300
94/94 [==============================] - ETA: 0s - loss: 0.1523
Epoch 250: val_loss did not improve from 0.14224
94/94 [==============================] - 69s 738ms/step - loss: 0.1523 - val_loss: 0.1483
Epoch 251/300
94/94 [==============================] - ETA: 0s - loss: 0.1526
Epoch 251: val_loss did not improve from 0.14224
94/94 [==============================] - 71s 764ms/step - loss: 0.1526 - val_loss: 0.1473
Epoch 252/300
94/94 [==============================] - ETA: 0s - loss: 0.1525
Epoch 252: val_loss did not improve from 0.14224
94/94 [==============================] - 85s 908ms/step - loss: 0.1525 - val_loss: 0.1481
Epoch 253/300
94/94 [==============================] - ETA: 0s - loss: 0.1520
Epoch 253: val_loss did not improve from 0.14224
94/94 [==============================] - 84s 895ms/step - loss: 0.1520 - val_loss: 0.1439
Epoch 254/300
94/94 [==============================] - ETA: 0s - loss: 0.1510
Epoch 254: val_loss improved from 0.14224 to 0.14202, 

Epoch 287/300
94/94 [==============================] - ETA: 0s - loss: 0.1452
Epoch 287: val_loss did not improve from 0.13541
94/94 [==============================] - 83s 883ms/step - loss: 0.1452 - val_loss: 0.1373
Epoch 288/300
94/94 [==============================] - ETA: 0s - loss: 0.1452
Epoch 288: val_loss did not improve from 0.13541
94/94 [==============================] - 82s 878ms/step - loss: 0.1452 - val_loss: 0.1477
Epoch 289/300
94/94 [==============================] - ETA: 0s - loss: 0.1460
Epoch 289: val_loss did not improve from 0.13541
94/94 [==============================] - 81s 866ms/step - loss: 0.1460 - val_loss: 0.1366
Epoch 290/300
94/94 [==============================] - ETA: 0s - loss: 0.1447
Epoch 290: val_loss did not improve from 0.13541
94/94 [==============================] - 67s 711ms/step - loss: 0.1447 - val_loss: 0.1432
Epoch 291/300
94/94 [==============================] - ETA: 0s - loss: 0.1511
Epoch 291: val_loss did not improve from 0.13541
94/94

In [10]:
# import tensorflow as tf
# from tensorflow.keras.models import load_model

# # Load the previously saved model
# model_path = 'transformer_model_hmf.h5'
# model = load_model(model_path)

In [11]:
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# # Re-compile the model with potentially adjusted learning rate or optimizer
# model.compile(optimizer=Adam(learning_rate=0.0001), loss='mape')

In [12]:
# # Callbacks for early stopping and saving the best model
# callback = EarlyStopping(monitor="val_loss", patience=50, verbose=1, mode="min")
# checkpoint = ModelCheckpoint(filepath='transformer_model_updated.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# # Assume you have your training and validation data loaded: x1_train, y1_train, x1_val, y1_val
# # Continue training the model
# history = model.fit(x1_train, y1_train, callbacks=[callback, checkpoint], batch_size=32, epochs=300, verbose=1, validation_data=(x1_val, y1_val), shuffle=False)

Epoch 1/300
375/375 [==============================] - ETA: 0s - loss: 0.1382
Epoch 1: val_loss improved from inf to 0.12950, saving model to transformer_model_updated.h5
375/375 [==============================] - 52s 97ms/step - loss: 0.1382 - val_loss: 0.1295
Epoch 2/300
375/375 [==============================] - ETA: 0s - loss: 0.1380
Epoch 2: val_loss did not improve from 0.12950
375/375 [==============================] - 61s 163ms/step - loss: 0.1380 - val_loss: 0.1297
Epoch 3/300
375/375 [==============================] - ETA: 0s - loss: 0.1375
Epoch 3: val_loss improved from 0.12950 to 0.12918, saving model to transformer_model_updated.h5
375/375 [==============================] - 80s 213ms/step - loss: 0.1375 - val_loss: 0.1292
Epoch 4/300
374/375 [============================>.] - ETA: 0s - loss: 0.1379
Epoch 4: val_loss did not improve from 0.12918
375/375 [==============================] - 75s 201ms/step - loss: 0.1379 - val_loss: 0.1299
Epoch 5/300
375/375 [================

375/375 [==============================] - ETA: 0s - loss: 0.1318
Epoch 72: val_loss improved from 0.12362 to 0.12323, saving model to transformer_model_updated.h5
375/375 [==============================] - 68s 180ms/step - loss: 0.1318 - val_loss: 0.1232
Epoch 73/300
375/375 [==============================] - ETA: 0s - loss: 0.1316
Epoch 73: val_loss did not improve from 0.12323
375/375 [==============================] - 68s 181ms/step - loss: 0.1316 - val_loss: 0.1244
Epoch 74/300
375/375 [==============================] - ETA: 0s - loss: 0.1321
Epoch 74: val_loss did not improve from 0.12323
375/375 [==============================] - 71s 190ms/step - loss: 0.1321 - val_loss: 0.1253
Epoch 75/300
375/375 [==============================] - ETA: 0s - loss: 0.1318
Epoch 75: val_loss did not improve from 0.12323
375/375 [==============================] - 69s 185ms/step - loss: 0.1318 - val_loss: 0.1260
Epoch 76/300
375/375 [==============================] - ETA: 0s - loss: 0.1319
Epoch 76

375/375 [==============================] - 86s 230ms/step - loss: 0.1294 - val_loss: 0.1206
Epoch 108/300
375/375 [==============================] - ETA: 0s - loss: 0.1291
Epoch 108: val_loss did not improve from 0.12064
375/375 [==============================] - 92s 246ms/step - loss: 0.1291 - val_loss: 0.1214
Epoch 109/300
375/375 [==============================] - ETA: 0s - loss: 0.1288
Epoch 109: val_loss improved from 0.12064 to 0.12034, saving model to transformer_model_updated.h5
375/375 [==============================] - 90s 239ms/step - loss: 0.1288 - val_loss: 0.1203
Epoch 110/300
375/375 [==============================] - ETA: 0s - loss: 0.1287
Epoch 110: val_loss did not improve from 0.12034
375/375 [==============================] - 75s 199ms/step - loss: 0.1287 - val_loss: 0.1210
Epoch 111/300
375/375 [==============================] - ETA: 0s - loss: 0.1286
Epoch 111: val_loss did not improve from 0.12034
375/375 [==============================] - 91s 242ms/step - loss: 

Epoch 143/300
375/375 [==============================] - ETA: 0s - loss: 0.1250
Epoch 143: val_loss did not improve from 0.11747
375/375 [==============================] - 88s 234ms/step - loss: 0.1250 - val_loss: 0.1185
Epoch 144/300
375/375 [==============================] - ETA: 0s - loss: 0.1254
Epoch 144: val_loss did not improve from 0.11747
375/375 [==============================] - 86s 231ms/step - loss: 0.1254 - val_loss: 0.1177
Epoch 145/300
375/375 [==============================] - ETA: 0s - loss: 0.1248
Epoch 145: val_loss did not improve from 0.11747
375/375 [==============================] - 86s 228ms/step - loss: 0.1248 - val_loss: 0.1176
Epoch 146/300
374/375 [============================>.] - ETA: 5s - loss: 0.1244 
Epoch 146: val_loss improved from 0.11747 to 0.11640, saving model to transformer_model_updated.h5
375/375 [==============================] - 2043s 5s/step - loss: 0.1244 - val_loss: 0.1164
Epoch 147/300
375/375 [==============================] - ETA: 0s -

375/375 [==============================] - ETA: 0s - loss: 0.1104
Epoch 211: val_loss improved from 0.10181 to 0.10065, saving model to transformer_model_updated.h5
375/375 [==============================] - 86s 230ms/step - loss: 0.1104 - val_loss: 0.1006
Epoch 212/300
375/375 [==============================] - ETA: 0s - loss: 0.1098
Epoch 212: val_loss did not improve from 0.10065
375/375 [==============================] - 86s 230ms/step - loss: 0.1098 - val_loss: 0.1010
Epoch 213/300
375/375 [==============================] - ETA: 0s - loss: 0.1101
Epoch 213: val_loss improved from 0.10065 to 0.10033, saving model to transformer_model_updated.h5
375/375 [==============================] - 82s 219ms/step - loss: 0.1101 - val_loss: 0.1003
Epoch 214/300
375/375 [==============================] - ETA: 0s - loss: 0.1095
Epoch 214: val_loss improved from 0.10033 to 0.09926, saving model to transformer_model_updated.h5
375/375 [==============================] - 85s 226ms/step - loss: 0.1095

Epoch 245/300
375/375 [==============================] - ETA: 0s - loss: 0.1003
Epoch 245: val_loss did not improve from 0.09101
375/375 [==============================] - 94s 252ms/step - loss: 0.1003 - val_loss: 0.0914
Epoch 246/300
375/375 [==============================] - ETA: 0s - loss: 0.1003
Epoch 246: val_loss did not improve from 0.09101
375/375 [==============================] - 94s 251ms/step - loss: 0.1003 - val_loss: 0.0928
Epoch 247/300
375/375 [==============================] - ETA: 0s - loss: 0.1001
Epoch 247: val_loss improved from 0.09101 to 0.09071, saving model to transformer_model_updated.h5
375/375 [==============================] - 89s 239ms/step - loss: 0.1001 - val_loss: 0.0907
Epoch 248/300
375/375 [==============================] - ETA: 0s - loss: 0.0997
Epoch 248: val_loss improved from 0.09071 to 0.08956, saving model to transformer_model_updated.h5
375/375 [==============================] - 88s 236ms/step - loss: 0.0997 - val_loss: 0.0896
Epoch 249/300
37

375/375 [==============================] - 86s 228ms/step - loss: 0.0942 - val_loss: 0.0848
Epoch 280/300
375/375 [==============================] - ETA: 0s - loss: 0.0941
Epoch 280: val_loss did not improve from 0.08476
375/375 [==============================] - 84s 225ms/step - loss: 0.0941 - val_loss: 0.0856
Epoch 281/300
375/375 [==============================] - ETA: 0s - loss: 0.0942
Epoch 281: val_loss did not improve from 0.08476
375/375 [==============================] - 84s 224ms/step - loss: 0.0942 - val_loss: 0.0868
Epoch 282/300
375/375 [==============================] - ETA: 0s - loss: 0.0934
Epoch 282: val_loss did not improve from 0.08476
375/375 [==============================] - 86s 229ms/step - loss: 0.0934 - val_loss: 0.0862
Epoch 283/300
375/375 [==============================] - ETA: 0s - loss: 0.0938
Epoch 283: val_loss did not improve from 0.08476
375/375 [==============================] - 84s 224ms/step - loss: 0.0938 - val_loss: 0.0864
Epoch 284/300
375/375 [=

In [ ]:
# import pandas as pd

# # Save the training history
# hist_df = pd.DataFrame(history.history)
# hist_df.to_csv('history_updated.csv')

In [32]:
# import tensorflow as tf
# from tensorflow.keras.models import load_model

# # Load the previously saved model
# model_path = 'transformer_model_updated_1.h5'
# model = load_model(model_path)

# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# # Re-compile the model with potentially adjusted learning rate or optimizer
# model.compile(optimizer=Adam(learning_rate=0.0001), loss='mape')

# # Callbacks for early stopping and saving the best model
# callback = EarlyStopping(monitor="val_loss", patience=50, verbose=1, mode="min")
# checkpoint = ModelCheckpoint(filepath='transformer_model_updated_2.h5', save_best_only=True, monitor='val_loss', mode='min', verbose=1)

# # Assume you have your training and validation data loaded: x1_train, y1_train, x1_val, y1_val
# # Continue training the model
# history = model.fit(x1_train, y1_train, callbacks=[callback, checkpoint], batch_size=32, epochs=300, verbose=1, validation_data=(x1_val, y1_val), shuffle=False)

Epoch 1/300
375/375 [==============================] - ETA: 0s - loss: 0.0785
Epoch 1: val_loss improved from inf to 0.07044, saving model to transformer_model_updated_2.h5


C:\Users\mali19\AppData\Local\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


375/375 [==============================] - 96s 241ms/step - loss: 0.0785 - val_loss: 0.0704
Epoch 2/300
375/375 [==============================] - ETA: 0s - loss: 0.0784
Epoch 2: val_loss improved from 0.07044 to 0.07031, saving model to transformer_model_updated_2.h5
375/375 [==============================] - 95s 254ms/step - loss: 0.0784 - val_loss: 0.0703
Epoch 3/300
375/375 [==============================] - ETA: 0s - loss: 0.0787
Epoch 3: val_loss improved from 0.07031 to 0.06992, saving model to transformer_model_updated_2.h5
375/375 [==============================] - 94s 251ms/step - loss: 0.0787 - val_loss: 0.0699
Epoch 4/300
375/375 [==============================] - ETA: 0s - loss: 0.0786
Epoch 4: val_loss improved from 0.06992 to 0.06943, saving model to transformer_model_updated_2.h5
375/375 [==============================] - 95s 253ms/step - loss: 0.0786 - val_loss: 0.0694
Epoch 5/300
375/375 [==============================] - ETA: 0s - loss: 0.0788
Epoch 5: val_loss did n

Epoch 38/300
375/375 [==============================] - ETA: 0s - loss: 0.0782
Epoch 38: val_loss did not improve from 0.06888
375/375 [==============================] - 56s 149ms/step - loss: 0.0782 - val_loss: 0.0689
Epoch 39/300
374/375 [============================>.] - ETA: 0s - loss: 0.0781
Epoch 39: val_loss did not improve from 0.06888
375/375 [==============================] - 54s 145ms/step - loss: 0.0781 - val_loss: 0.0692
Epoch 40/300
375/375 [==============================] - ETA: 0s - loss: 0.0781
Epoch 40: val_loss did not improve from 0.06888
375/375 [==============================] - 55s 147ms/step - loss: 0.0781 - val_loss: 0.0700
Epoch 41/300
374/375 [============================>.] - ETA: 0s - loss: 0.0780
Epoch 41: val_loss did not improve from 0.06888
375/375 [==============================] - 66s 175ms/step - loss: 0.0780 - val_loss: 0.0700
Epoch 42/300
375/375 [==============================] - ETA: 0s - loss: 0.0777
Epoch 42: val_loss did not improve from 0.068

In [15]:
# import pandas as pd

# # Save the training history
# hist_df = pd.DataFrame(history.history)
# hist_df.to_csv('history_updated_2.csv')

In [6]:
import numpy as np
import scipy
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras import backend
from matplotlib import pyplot as plt
import joblib

samples = 200000
frames_per_file = 1
seq_length = 25 #sequence window to be used
steps_to_pred = 50000 #number of steps ahead that want to be predicted
batch_size = 128
molecules = 2
species = 'hmf'
# horizons = np.array([1, 50, 100, 200, 300])
horizons = np.array([100,100000])
downsample_factor = 10
samples = int(samples/downsample_factor)
steps_to_pred = int(steps_to_pred/downsample_factor)

def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(samples-seq_length):
        x = data[i:(i+seq_length), :]
        y = data[i+seq_length, :]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)




In [7]:
data_1 = np.load('C:/Users/mali19/Downloads/KevinCode/inputs/input_latest/%s_data.npy' %species)
data = np.add(data_1, 3.549)
data = data[::downsample_factor, :]

print(data.shape)


(20000, 6)


In [8]:
input_, output = create_sequences(data, seq_length) #(samples-seq_length) x seq_length x 8000, (samples-seq_length) x 8000 tensor

# model = load_model('transformer_model_%s.h5' % species)

model = load_model('transformer_model_hmf.h5')


In [13]:
horizons = [5000,10000]

In [14]:
for j in range(2):
    horizon = horizons[j]

    #(samples-seq_length) x seq_length x 8002, (samples-seq_length) x 8001 tensor
    test = np.expand_dims(input_[-steps_to_pred, :, :], axis=0)

    performance = np.zeros((steps_to_pred, 1))
    predictions = np.zeros((steps_to_pred, 2*molecules*3))

    for i in range(steps_to_pred):
        prediction = model.predict(test, steps=1, verbose=0)
        temp = np.expand_dims(output[-steps_to_pred+i, :], axis=0)

        performance[i, 0] = tf.metrics.mean_absolute_percentage_error(temp, prediction)

        predictions[i, 0:molecules*3] = np.expand_dims(prediction, axis=0)
        predictions[i, molecules*3:2*molecules*3] = temp

        test = np.append(test[:,1:,:], np.expand_dims(prediction, axis=0), axis=1)

        if horizon != 1:
            if i % horizon != 0:
                test = np.append(test[:,1:,:], np.expand_dims(prediction, axis=0), axis=1)

            else:
                test = np.expand_dims(input_[-steps_to_pred+i+1, :, :], axis=0)
        else:
            test = np.expand_dims(input_[-steps_to_pred+i+1, :, :], axis=0)

        if i % 1000 == 0:
            print("Step %i predicted" %i)

    np.savetxt("11performance_%s_horizon_%s.csv" %(species, horizon), performance, delimiter=",")
    np.savetxt("11predictions_%s_horizon_%s.csv" %(species, horizon), predictions, delimiter=",")

#np.savetxt("11performance_%s.csv" %species, performance, delimiter=",")
#np.savetxt("11predictions_%s.csv" %species, predictions, delimiter=",")

Step 0 predicted
Step 1000 predicted
Step 2000 predicted
Step 3000 predicted
Step 4000 predicted
Step 0 predicted
Step 1000 predicted
Step 2000 predicted
Step 3000 predicted
Step 4000 predicted
